In [1]:
from numpy import cos, sin, arccos, arcsin
import pybullet as p
import numpy as np

class Camera():
    @staticmethod
    def camera_upvec(pos_vec):
        theta = np.arccos(pos_vec[-1])
        sintheta = np.sin(theta)
        phi = np.arccos(pos_vec[0]/sintheta) # fails if sintheta = 0

        u1 = np.array([cos(theta)*cos(phi), cos(theta)*sin(phi), -sin(theta)])
        # u2 = np.array([-sin(phi), cos(phi), 0])

        return -u1
    
    def reset_view(self, pos=None, target=None, up_vec=None):
        if pos is not None:
            self.pos_vec = np.array(pos)
            if up_vec is None:
                self.up_vec = self.camera_upvec(self.pos_vec)
            else:
                self.up_vec = np.array(up_vec)
            
        if target is not None:
            self.target = np.array(target)
            
        self.viewMat = p.computeViewMatrix(
            cameraEyePosition=self.pos_vec,
            cameraTargetPosition=self.target,
            cameraUpVector=self.up_vec)
    
    def __init__(self, pos, target=[0,0,0], up_vec=None):
        self.reset_view(pos=pos, target=target, up_vec=up_vec)
    
        self.width = 224
        self.height = 224
        
        self.viewMat = p.computeViewMatrix(
            cameraEyePosition=self.pos_vec,
            cameraTargetPosition=self.target,
            cameraUpVector=self.up_vec)

        self.projMat = p.computeProjectionMatrixFOV(
            fov=70.25,
            aspect=1.0,
            nearVal=0,
            farVal=3.0)

    def get_image(self, pos=None, target=None):
        self.reset_view(pos=pos, target=target)
        
        width, height, rgbImg, depthImg, segImg = p.getCameraImage(
            width=self.width, 
            height=self.height,
            viewMatrix=self.viewMat,
            projectionMatrix=self.projMat)
        
        return rgbImg, depthImg, segImg
    
    def get_point_cloud(pos=None, target=None):
        # get a depth image
        # "infinite" depths will have a value close to 1
        image_arr = self.get_image(pos=pos, target=target)
        depth = image_arr[3]

        # create a 4x4 transform matrix that goes from pixel coordinates (and depth values) to world coordinates
        proj_matrix = np.asarray(self.proj_matrix).reshape([4, 4], order="F")
        view_matrix = np.asarray(self.view_matrix).reshape([4, 4], order="F")
        tran_pix_world = np.linalg.inv(np.matmul(proj_matrix, view_matrix))

        # create a grid with pixel coordinates and depth values
        y, x = np.mgrid[-1:1:2 / self.height, -1:1:2 / self.width]
        y *= -1.
        x, y, z = x.reshape(-1), y.reshape(-1), depth.reshape(-1)
        h = np.ones_like(z)

        pixels = np.stack([x, y, z, h], axis=1)
        # filter out "infinite" depths
        pixels = pixels[z < 0.99]
        pixels[:, 2] = 2 * pixels[:, 2] - 1

        # turn pixels to world coordinates
        points = np.matmul(tran_pix_world, pixels.T).T
        points /= points[:, 3: 4]
        points = points[:, :3]

        return points

pybullet build time: Nov 30 2022 16:56:59


In [2]:
import pybullet as p
import time
import pybullet_data
physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally
p.setGravity(0,0,-10)
planeId = p.loadURDF("plane.urdf")
startPos = [0,0,1]
startOrientation = p.getQuaternionFromEuler([0,0,0])
boxId = p.loadURDF("r2d2.urdf",startPos, startOrientation)
#set the center of mass frame (loadURDF sets base link frame) startPos/Ornp.resetBasePositionAndOrientation(boxId, startPos, startOrientation)
for i in range (500):
    p.stepSimulation()
    time.sleep(1./240.)
cubePos, cubeOrn = p.getBasePositionAndOrientation(boxId)
print(cubePos,cubeOrn)

# print('making camera')

# cam1 = Camera([0, 0, 2], up_vec=[0, 1, 0])

# print('getting camera')

# cam1.get_image()

# print('got image')

# p.disconnect()

# viewMat = [
#     0.642787516117096, -0.4393851161003113, 0.6275069713592529, 0.0, 0.766044557094574,
#     0.36868777871131897, -0.5265407562255859, 0.0, -0.0, 0.8191521167755127, 0.5735764503479004,
#     0.0, 2.384185791015625e-07, 2.384185791015625e-07, -5.000000476837158, 1.0
# ]
viewMat = p.computeViewMatrix(
            cameraEyePosition=[0,0,1.5],
            cameraTargetPosition=[0,0,0],
            cameraUpVector=[0,1,0])

projMat = [
    0.7499999403953552, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, -1.0000200271606445, -1.0,
    0.0, 0.0, -0.02000020071864128, 0.0
]

'''
self.fov = 90
        self.aspect_ratio = 640./480
        self.max_camera_distance = 6
        self.pixel_width = 640
        self.pixel_height = 480
        self.near_val = 0.6
        '''

projMat = p.computeProjectionMatrixFOV(
        fov=120,
        aspect=1.6,
        nearVal=0.01,
        farVal=30)

width = 224
height = 224

images = p.getCameraImage(width,
                          height,
                          viewMatrix=viewMat,
                          projectionMatrix=projMat,
                         )
proj_opengl = np.reshape(images[2], (height, width, 4)) * 1. / 255.
time.sleep(1)


(1.6512402958188955e-07, 1.772747713351012e-05, 0.4709923057372396) (-2.835336350240825e-08, 4.7804928448822215e-09, 1.2080008391170977e-06, 0.9999999999992699)
